<a href="https://colab.research.google.com/github/astrissha/Heart-Disease-Prediction/blob/main/HeartDiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python tensorflow numpy scikit-learn tqdm


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Parameters
DATASET_PATH = '/content/drive/MyDrive/HeartDiseaseDataset'
IMG_SIZE = 64  # Resize video frames to 64x64
SEQUENCE_LENGTH = 30  # Number of frames per video used
CATEGORIES = ['Normal', 'Diseased']

def load_videos():
    data = []
    labels = []

    for label, category in enumerate(CATEGORIES):
        category_path = os.path.join(DATASET_PATH, category)
        for video_file in tqdm(os.listdir(category_path), desc=f"Loading {category} videos"):
            video_path = os.path.join(category_path, video_file)
            frames = extract_frames(video_path)
            if frames is not None:
                data.append(frames)
                labels.append(label)

    return np.array(data), to_categorical(labels, num_classes=2)

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Frame skipping to get SEQUENCE_LENGTH uniformly spaced frames
    step = max(1, total_frames // SEQUENCE_LENGTH)

    for i in range(SEQUENCE_LENGTH):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame / 255.0  # Normalize
        frames.append(frame)

    cap.release()

    # If we don’t get enough frames, discard the video
    if len(frames) == SEQUENCE_LENGTH:
        return np.array(frames)
    else:
        return None

# Load and prepare the data
X, y = load_videos()
print("Data shape:", X.shape)  # (samples, sequence_len, height, width, channels)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN + LSTM model
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),

    LSTM(64),  # LSTM layer after CNN processing
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes: Normal and Diseased
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")


Loading Diseased videos: 100%|██████████| 1/1 [00:11<00:00, 11.71s/it]
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Data shape: (3, 30, 64, 64, 3)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 30, 62, 62, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 31, 31, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 29, 29, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 30, 14, 14, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 30, 12544)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │     3,227,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,251,586 (12.40 MB)

 Trainable params: 3,251,586 (12.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.5000 - loss: 0.6828 - val_accuracy: 0.0000e+00 - val_loss: 1.6824
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.7825 - val_accuracy: 0.0000e+00 - val_loss: 1.1032
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.3352 - val_accuracy: 1.0000 - val_loss: 0.5369
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.1708 - val_accuracy: 1.0000 - val_loss: 0.3106
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.1080 - val_accuracy: 1.0000 - val_loss: 0.2158
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0620 - val_accuracy: 1.0000 - val_loss: 0.1605
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0389 - val_accuracy: 1.0000 - val_loss: 0.1278
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0266 - val_accuracy: 1.0000 - val_loss: 0.1058
Epoch 

In [3]:
# Save model
model.save("cardiac_model.h5")

# Load model later
# from tensorflow.keras.models import load_model
# model = load_model("cardiac_model.h5")


In [4]:
def predict_on_video(video_path):
    frames = extract_frames(video_path)
    if frames is None:
        print("Not enough frames in video.")
        return

    frames = np.expand_dims(frames, axis=0)
    pred = model.predict(frames)
    result = np.argmax(pred)
    print(f"Prediction: {'Normal' if result == 0 else 'Diseased'}")

# Example
# predict_on_video("test_video.mp4")


In [8]:
%%writefile Heart.py
import streamlit as st
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model

# Load model
model = load_model("cardiac_model.h5")

# Constants
IMG_SIZE = 64
SEQUENCE_LENGTH = 30
CATEGORIES = ['Normal', 'Diseased']

# Extract frames function
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // SEQUENCE_LENGTH)

    for i in range(SEQUENCE_LENGTH):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame / 255.0
        frames.append(frame)

    cap.release()

    if len(frames) == SEQUENCE_LENGTH:
        return np.array(frames)
    else:
        return None

# Prediction function
def predict_video(video_file):
    video_path = "temp_video.mp4"
    with open(video_path, "wb") as f:
        f.write(video_file.read())

    frames = extract_frames(video_path)
    os.remove(video_path)

    if frames is None:
        return "Error: Not enough frames in video."

    frames = np.expand_dims(frames, axis=0)  # (1, 30, 64, 64, 3)
    prediction = model.predict(frames)[0]
    class_index = np.argmax(prediction)
    confidence = prediction[class_index] * 100
    return f"Prediction: **{CATEGORIES[class_index]}** ({confidence:.2f}% confidence)"

# Streamlit App UI
st.set_page_config(page_title="Cardiac Cycle Detector", layout="centered")
st.title("🫀 Cardiac Cycle Analysis for Heart Disease Detection")
st.markdown("Upload an echocardiographic video (.mp4) to check if it shows signs of heart disease.")

video_file = st.file_uploader("Upload Video", type=["mp4"])

if video_file is not None:
    with st.spinner("Analyzing video..."):
        result = predict_video(video_file)
    st.success(result)


Writing Heart.py


In [6]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
